In [1]:
import os 
os.chdir('../')
%pwd

'd:\\Heart_Ml'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [3]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.SVC
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [5]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,accuracy_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [6]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        acc_score=accuracy_score(actual, pred)
        return acc_score

    
    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)

        acc_score = self.eval_metrics(test_y, predicted_qualities)
        
        # Saving metrics as local
        scores = {"accuracy_score":acc_score}
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [7]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-09-02 00:59:27,401: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-02 00:59:27,410: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-02 00:59:27,412: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-02 00:59:27,413: INFO: common: created directory at: artifacts]
[2024-09-02 00:59:27,416: INFO: common: created directory at: artifacts/model_evaluation]
[2024-09-02 00:59:27,482: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
